### Concurso predecir compra de casas.

En este modelo, eliminaremos algunas de las variables que no son significativas según la rueba de valor P hecha con anterioridad 

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import optuna 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import models, layers 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras.optimizers import RMSprop, SGD, Adam
from tensorflow.keras import regularizers
import matplotlib.pyplot as plt
import statsmodels.api as sm

In [3]:
import mlflow
import mlflow.tensorflow
mlflow.tensorflow.autolog()
import plotly.io as pio

In [5]:
import os
from getpass import getpass

#vinculamos la cuenta de dagshub con mlflow
os.environ['MLFLOW_TRACKING_USERNAME'] = "mseguracarrillo7"
os.environ['MLFLOW_TRACKING_PASSWORD'] = getpass("Enter your DAGsHub access token or password: ")
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/mseguracarrillo7/Prediccion_CompraC"
mlflow.set_tracking_uri("https://dagshub.com/mseguracarrillo7/Prediccion_CompraC.mlflow")

Enter your DAGsHub access token or password:  ········


In [72]:
data= pd.DataFrame(pd.read_csv("global_house_purchase_dataset.csv"))

In [74]:
paises_unicos = data['country'].unique() #países en la base de datos 

print("Los países en el dataset son:")
print(paises_unicos)

Los países en el dataset son:
['France' 'South Africa' 'Germany' 'Canada' 'Brazil' 'UAE' 'Australia'
 'UK' 'USA' 'China' 'Singapore' 'India' 'Japan']


In [76]:
# Tasas de conversión aproximadas a USD (Octubre 2025)
# 1 Moneda Local = X Dólares USD
rates_to_usd = {
    'France': 1.08,        # Euro (EUR)
    'South Africa': 0.053,   # Rand (ZAR)
    'Germany': 1.08,       # Euro (EUR)
    'Canada': 0.73,        # Dólar Canadiense (CAD)
    'Brazil': 0.18,        # Real Brasileño (BRL)
    'UAE': 0.27,           # Dírham de los EAU (AED)
    'Australia': 0.66,     # Dólar Australiano (AUD)
    'UK': 1.25,            # Libra Esterlina (GBP)
    'USA': 1.0,            # Dólar Estadounidense (USD)
    'China': 0.14,         # Yuan Chino (CNY)
    'Singapore': 0.74,     # Dólar de Singapur (SGD)
    'India': 0.012,        # Rupia India (INR)
    'Japan': 0.0064       # Yen Japonés (JPY)
}

In [78]:
data['price_USD'] = data.apply(
    lambda row: row['price'] * rates_to_usd.get(row['country'], 1.0), 
    axis=1
) #creamos una nueva columna de precio de la casa en la que aplicamos la conversión según el país 

#ahora haremos lo mismo para las columnas (customer_salary, loan_amount, monthly_expenses, down_payment )
data['customer_salary_USD'] = data.apply(
    lambda row: row['customer_salary'] * rates_to_usd.get(row['country'], 1.0), 
    axis=1
)

data['loan_amount_USD'] = data.apply(
    lambda row: row['loan_amount'] * rates_to_usd.get(row['country'], 1.0), 
    axis=1
)

data['monthly_expenses_USD'] = data.apply(
    lambda row: row['monthly_expenses'] * rates_to_usd.get(row['country'], 1.0), 
    axis=1
)

data['down_payment_USD'] = data.apply(
    lambda row: row['down_payment'] * rates_to_usd.get(row['country'], 1.0), 
    axis=1
)

#borramos las columnas iniciales 
data = data.drop(['price', 'customer_salary', 'loan_amount', 'monthly_expenses', 'down_payment'], axis=1)


Ahora borraremos algunas columnas que no tienen mucha influencia en el modelo, como la identificación que se le da a cada dato, la ciudad a la que pertenece la casa, los casos legales, baños y garage. 

In [45]:
x = data.drop(["decision", 'property_id', 'city', 'legal_cases_on_property', 'bathrooms', 'garage'],axis=1)

In [47]:
x = pd.get_dummies(x, drop_first=True) 

In [49]:
scaler = StandardScaler()
x_scaled = pd.DataFrame(scaler.fit_transform(x), columns=x.columns) #aplicamos la normalizaciómn a las variables en x

In [51]:
y = data['decision']

In [53]:
x_train, x_Test, y_train, y_Test = train_test_split(x_scaled,y, train_size=0.7, random_state=123)

In [55]:
x_test, x_val, y_test, y_val = train_test_split(x_Test,y_Test, train_size=2/3, random_state=123)

Buscaremos nuevos hiperparámetros en Optuna para este nuevo modelo.

In [58]:
def create_model(trial):
    with mlflow.start_run(nested=True): 
        params= {
            "learning_rate": trial.suggest_float("learning_rate",1e-4, 1e-2, log=True),
            "n_layers": trial.suggest_int("n_layers",1,3),
            "dropout_rate": trial.suggest_float("dropout_rate", 0.1, 0.5),
            "optimizer": trial.suggest_categorical("optimizer", ["adam", "rmsprop", "sgd"])
        }

        mlflow.log_params(params)
        model = models.Sequential()
        model.add(tf.keras.layers.Input(shape=(x_train.shape[1],))) 
    
        for i in range (params["n_layers"]):
            n_units = trial.suggest_int(f"n_units_l{i}", 32, 256)
            model.add(layers.Dense(n_units, activation = 'relu'))
            model.add(tf.keras.layers.Dropout(params["dropout_rate"])) #aplicamos el dropout sugerido
            
            mlflow.log_param(f"n_units_l{i}", n_units)
        model.add(layers.Dense(1, activation='sigmoid'))

        lr = params["learning_rate"]
        optimizer_selected = params["optimizer"]

        if optimizer_selected == "adam":
            optimizer = tf.keras.optimizers.Adam(learning_rate = lr)
        elif optimizer_selected == "sgd":
            optimizer = tf.keras.optimizers.SGD(learning_rate = lr)
        elif optimizer_selected == "rmsprop":
            optimizer = tf.keras.optimizers.RMSprop(learning_rate = lr)

    #usaremos binary cross entropy porque solo tenemos dos pposibles salidas (compro, no compro)
        model.compile(optimizer = optimizer,
                      loss = "binary_crossentropy",
                      metrics = ["accuracy"])
    return model 

mlflow.keras.autolog()

In [60]:
def objective(trial):
    modelo= create_model(trial)
    history= modelo.fit(x_train, y_train, epochs=5, batch_size=64, verbose=0)
    loss, accuracy = modelo.evaluate(x_test, y_test, verbose=0)
    mlflow.log_metric("accuracy", accuracy)
    return accuracy 

In [62]:
#prueba sin algunas variables.
study = optuna.create_study(direction = "maximize")
study.optimize(objective, n_trials=30)

[I 2025-10-10 20:19:01,891] A new study created in memory with name: no-name-60e0d80e-52e8-46a4-afd8-de3d7dc5616c


🏃 View run charming-fish-552 at: https://dagshub.com/mseguracarrillo7/Prediccion_CompraC.mlflow/#/experiments/0/runs/cc106d82551a48f686f2a291092c94d6
🧪 View experiment at: https://dagshub.com/mseguracarrillo7/Prediccion_CompraC.mlflow/#/experiments/0


2025/10/10 20:19:09 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '2cfafde4aab048e0af6a851ec2fc74da', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current keras workflow


🏃 View run gregarious-skink-240 at: https://dagshub.com/mseguracarrillo7/Prediccion_CompraC.mlflow/#/experiments/0/runs/2cfafde4aab048e0af6a851ec2fc74da
🧪 View experiment at: https://dagshub.com/mseguracarrillo7/Prediccion_CompraC.mlflow/#/experiments/0


2025/10/10 20:19:51 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during keras autologging: INTERNAL_ERROR: Response: {'error': 'unsupported endpoint, please contact support@dagshub.com'}
[I 2025-10-10 20:19:55,627] Trial 0 finished with value: 0.9225249886512756 and parameters: {'learning_rate': 0.00046825622495482296, 'n_layers': 2, 'dropout_rate': 0.19332813091419765, 'optimizer': 'rmsprop', 'n_units_l0': 111, 'n_units_l1': 98}. Best is trial 0 with value: 0.9225249886512756.


...

2025/10/10 20:53:27 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during keras autologging: INTERNAL_ERROR: Response: {'error': 'unsupported endpoint, please contact support@dagshub.com'}
[I 2025-10-10 20:53:33,630] Trial 27 finished with value: 0.9237750172615051 and parameters: {'learning_rate': 0.0005595973706178091, 'n_layers': 2, 'dropout_rate': 0.33762868934472257, 'optimizer': 'adam', 'n_units_l0': 229, 'n_units_l1': 117}. Best is trial 4 with value: 0.9239000082015991.


🏃 View run colorful-rook-809 at: https://dagshub.com/mseguracarrillo7/Prediccion_CompraC.mlflow/#/experiments/0/runs/082ba06ad0144b33962c06c24a87ecc0
🧪 View experiment at: https://dagshub.com/mseguracarrillo7/Prediccion_CompraC.mlflow/#/experiments/0


2025/10/10 20:54:25 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during keras autologging: INTERNAL_ERROR: Response: {'error': 'unsupported endpoint, please contact support@dagshub.com'}
[I 2025-10-10 20:54:32,221] Trial 28 finished with value: 0.9207000136375427 and parameters: {'learning_rate': 0.001040193555171342, 'n_layers': 3, 'dropout_rate': 0.25738648172087436, 'optimizer': 'rmsprop', 'n_units_l0': 199, 'n_units_l1': 145, 'n_units_l2': 114}. Best is trial 4 with value: 0.9239000082015991.


🏃 View run melodic-dove-989 at: https://dagshub.com/mseguracarrillo7/Prediccion_CompraC.mlflow/#/experiments/0/runs/87ebc86cfa1948e9b99cd2923aaf6b31
🧪 View experiment at: https://dagshub.com/mseguracarrillo7/Prediccion_CompraC.mlflow/#/experiments/0


2025/10/10 20:55:26 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during keras autologging: INTERNAL_ERROR: Response: {'error': 'unsupported endpoint, please contact support@dagshub.com'}
[I 2025-10-10 20:55:33,843] Trial 29 finished with value: 0.7955999970436096 and parameters: {'learning_rate': 0.0004835355658049402, 'n_layers': 1, 'dropout_rate': 0.31218784652007325, 'optimizer': 'sgd', 'n_units_l0': 101}. Best is trial 4 with value: 0.9239000082015991.


No podemos esperar un muy buen resultado como el de la primera prueba, entonces no ha favorecido al modelo quitar las variables que hemos quitado, podemos probar con otras, aunque primero veamos el R-squared.

In [66]:
#creación del modelo
x_train = sm.add_constant(x_train, prepend=True)
modelo = sm.Logit(endog = y_train, exog = x_train)
modelo_fit = modelo.fit()
print(modelo_fit.summary())

Optimization terminated successfully.
         Current function value: 0.280878
         Iterations 8
                           Logit Regression Results                           
Dep. Variable:               decision   No. Observations:               140000
Model:                          Logit   Df Residuals:                   139965
Method:                           MLE   Df Model:                           34
Date:                Fri, 10 Oct 2025   Pseudo R-squ.:                  0.4803
Time:                        20:56:24   Log-Likelihood:                -39323.
converged:                       True   LL-Null:                       -75658.
Covariance Type:            nonrobust   LLR p-value:                     0.000
                                       coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
const                               -2.7946      0.017   -166.68

Tenemos un R-squared de 0.48 lo cual no es muy bueno, probemos con otras variables

In [107]:
x = data.drop(["decision", 'property_id', 'city'],axis=1)

In [109]:
x = pd.get_dummies(x, drop_first=True) 

In [111]:
scaler = StandardScaler()
x_scaled = pd.DataFrame(scaler.fit_transform(x), columns=x.columns) #aplicamos la normalizaciómn a las variables en x

In [113]:
y= data['decision']

In [115]:
x_train, x_Test, y_train, y_Test = train_test_split(x_scaled,y, train_size=0.7, random_state=123)
x_test, x_val, y_test, y_val = train_test_split(x_Test,y_Test, train_size=2/3, random_state=123)

In [102]:
#creación del modelo
x_train = sm.add_constant(x_train, prepend=True)
modelo = sm.Logit(endog = y_train, exog = x_train)
modelo_fit = modelo.fit()
print(modelo_fit.summary())

         Current function value: 0.128551
         Iterations: 35


C:\Users\Moises\anaconda3\envs\tf_env\lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


                           Logit Regression Results                           
Dep. Variable:               decision   No. Observations:               140000
Model:                          Logit   Df Residuals:                   139962
Method:                           MLE   Df Model:                           37
Date:                Fri, 10 Oct 2025   Pseudo R-squ.:                  0.7621
Time:                        21:00:39   Log-Likelihood:                -17997.
converged:                      False   LL-Null:                       -75658.
Covariance Type:            nonrobust   LLR p-value:                     0.000
                                       coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
const                              -13.9002   1.76e+05  -7.89e-05      1.000   -3.45e+05    3.45e+05
property_size_sqft                   0.6130      0.024     25.615

Mejoró bastante el R-squared (0.7621), es un buen modelo, tiene el mismo valor que considerando todas las variables, entonces probemos entrenar la red nneuronal con estas.

In [117]:
#prueba sin algunas variables.
study = optuna.create_study(direction = "maximize")
study.optimize(objective, n_trials=30)

[I 2025-10-10 21:04:32,569] A new study created in memory with name: no-name-0de4651f-fd18-4e69-9dc3-d291b9461bfa


🏃 View run welcoming-turtle-148 at: https://dagshub.com/mseguracarrillo7/Prediccion_CompraC.mlflow/#/experiments/0/runs/3ed36442b47943fea16723ada1bb645d
🧪 View experiment at: https://dagshub.com/mseguracarrillo7/Prediccion_CompraC.mlflow/#/experiments/0


2025/10/10 21:05:51 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during keras autologging: INTERNAL_ERROR: Response: {'error': 'unsupported endpoint, please contact support@dagshub.com'}
[I 2025-10-10 21:05:56,453] Trial 0 finished with value: 0.9503250122070312 and parameters: {'learning_rate': 0.0045805424347809915, 'n_layers': 2, 'dropout_rate': 0.3157175385945594, 'optimizer': 'sgd', 'n_units_l0': 145, 'n_units_l1': 243}. Best is trial 0 with value: 0.9503250122070312.


🏃 View run amazing-gnat-169 at: https://dagshub.com/mseguracarrillo7/Prediccion_CompraC.mlflow/#/experiments/0/runs/f8cf65d242a94f8a98c9b9407bb9c519
🧪 View experiment at: https://dagshub.com/mseguracarrillo7/Prediccion_CompraC.mlflow/#/experiments/0


2025/10/10 21:06:52 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during keras autologging: INTERNAL_ERROR: Response: {'error': 'unsupported endpoint, please contact support@dagshub.com'}
[I 2025-10-10 21:06:57,291] Trial 1 finished with value: 0.9929749965667725 and parameters: {'learning_rate': 0.0001586577057763614, 'n_layers': 2, 'dropout_rate': 0.24811371324293852, 'optimizer': 'rmsprop', 'n_units_l0': 121, 'n_units_l1': 94}. Best is trial 1 with value: 0.9929749965667725.


...

2025/10/10 21:39:47 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during keras autologging: INTERNAL_ERROR: Response: {'error': 'unsupported endpoint, please contact support@dagshub.com'}
[I 2025-10-10 21:39:52,710] Trial 27 finished with value: 0.9974750280380249 and parameters: {'learning_rate': 0.006426233904010512, 'n_layers': 2, 'dropout_rate': 0.20236444550947275, 'optimizer': 'rmsprop', 'n_units_l0': 158, 'n_units_l1': 166}. Best is trial 7 with value: 0.9988499879837036.


🏃 View run secretive-perch-304 at: https://dagshub.com/mseguracarrillo7/Prediccion_CompraC.mlflow/#/experiments/0/runs/57b5c85886844e1dac502eeaa78c0925
🧪 View experiment at: https://dagshub.com/mseguracarrillo7/Prediccion_CompraC.mlflow/#/experiments/0


2025/10/10 21:40:48 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during keras autologging: INTERNAL_ERROR: Response: {'error': 'unsupported endpoint, please contact support@dagshub.com'}
[I 2025-10-10 21:40:57,638] Trial 28 finished with value: 0.9396250247955322 and parameters: {'learning_rate': 0.0014924038854799858, 'n_layers': 2, 'dropout_rate': 0.29560112777708364, 'optimizer': 'sgd', 'n_units_l0': 192, 'n_units_l1': 66}. Best is trial 7 with value: 0.9988499879837036.


🏃 View run valuable-sheep-497 at: https://dagshub.com/mseguracarrillo7/Prediccion_CompraC.mlflow/#/experiments/0/runs/8549752f58df4397aad09bb2f3e3970a
🧪 View experiment at: https://dagshub.com/mseguracarrillo7/Prediccion_CompraC.mlflow/#/experiments/0


2025/10/10 21:42:33 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during keras autologging: INTERNAL_ERROR: Response: {'error': 'unsupported endpoint, please contact support@dagshub.com'}
[I 2025-10-10 21:42:40,590] Trial 29 finished with value: 0.9976249933242798 and parameters: {'learning_rate': 0.004241528493577921, 'n_layers': 3, 'dropout_rate': 0.33816247993035625, 'optimizer': 'adam', 'n_units_l0': 131, 'n_units_l1': 145, 'n_units_l2': 249}. Best is trial 7 with value: 0.9988499879837036.


Ahora, probaremos con los mejores hiperparámetros obtenidos.

- learning rate = 0.0051305776
- n_layers= 2
- dropout rate = 0.49334236200747184
- optimizer = RMSprop
- n_unitsl0 = 223
- n_unitsl1 = 88

In [123]:
modelf = tf.keras.models.Sequential()
modelf.add(tf.keras.layers.Input(shape=(x_train.shape[1],)))
modelf.add(tf.keras.layers.Dense(223, activation='relu'))
modelf.add(tf.keras.layers.Dropout(0.49334236200747184))
modelf.add(tf.keras.layers.Dense(88, activation='relu'))
modelf.add(tf.keras.layers.Dropout(0.49334236200747184))
modelf.add(tf.keras.layers.Dense(1, activation='sigmoid'))

optimizer = tf.keras.optimizers.RMSprop(learning_rate= 0.0051305776)

modelf.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
modelf.summary()

#entrenamos el modelo con el conjunto de entrenamiento
history= modelf.fit(x_train,y_train, epochs=40, batch_size=64, verbose=1, validation_data=(x_test, y_test))

Model: "sequential_61"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_178 (Dense)               │ (None, 223)            │         8,697 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_117 (Dropout)           │ (None, 223)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_179 (Dense)               │ (None, 88)             │        19,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_118 (Dropout)           │ (None, 88)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_180 (Dense)               │ (None, 1)              │            89 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 28,498 (111.32 KB)

 Trainable params: 28,498 (111.32 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/40
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 24s 8ms/step - accuracy: 0.9742 - loss: 0.0639 - val_accuracy: 0.9972 - val_loss: 0.0075
Epoch 2/40
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 17s 8ms/step - accuracy: 0.9949 - loss: 0.0163 - val_accuracy: 0.9973 - val_loss: 0.0068
Epoch 3/40
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 16s 7ms/step - accuracy: 0.9961 - loss: 0.0128 - val_accuracy: 0.9980 - val_loss: 0.0057
Epoch 4/40
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 16s 7ms/step - accuracy: 0.9965 - loss: 0.0120 - val_accuracy: 0.9978 - val_loss: 0.0051
Epoch 5/40
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 21s 7ms/step - accuracy: 0.9966 - loss: 0.0121 - val_accuracy: 0.9983 - val_loss: 0.0043
Epoch 6/40
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 15s 7ms/step - accuracy: 0.9970 - loss: 0.0110 - val_accuracy: 0.9974 - val_loss: 0.0067
Epoch 7/40
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 15s 7ms/step - accuracy: 0.9968 - loss: 0.0115 - val_accuracy: 0.9981 - val_loss: 0.0049
Epoch 8/40
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 16s 7ms/step - accuracy: 0.9971 - loss: 0

2025/10/11 19:46:06 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during keras autologging: INVALID_PARAMETER_VALUE: Response: {'error_code': 'INVALID_PARAMETER_VALUE'}


In [125]:
val_loss, val_accuracy = modelf.evaluate(x_val, y_val)
print(f"Precisión final: {val_accuracy*100:.2f}%")

625/625 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.9981 - loss: 0.0056
Precisión final: 99.81%


Tenemos una precisión un poco menor que en nuestra primera red, por lo tanto mi resultado principal será el de la primera con un 99.91% de precisión.